1 - Set up Kernel and Required Dependencies

In [ ]:
# pip install --upgrade pip   or python.exe -m pip install --upgrade pip
# pip install --disable-pip-version-check
#     torch==1.13.1
#     torchdata==0.5.1 --quiet   (ajuda no carregamento de dados e em alguns outros aspectos específicos para conjunto de dados)
# pip install
#     transformers==4.27.2 (é uma bilioteca da Hugging Face, uma empresa de código aberto que criou muitas ferramentas de código aberto para grandes modelos de linguagem)
#     datasets==2.11.0 --quiet (biblioteca python da Hugging Face, que pode ser carregado em muitos conjuntos de dados públicos que as pessoas usam para treinar modelos)

In [7]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

2 - Summarize Dialouge without Prompt Engineering

Neste caso de uso, vamos gerar resumo de um diálogo com o Large Language Model (LLM) FLAN-TS pré-treinado da Hugging Face. A lista de modelos disponíveis no pacote de transformadores da Hugging Face pode ser encontrada aqui.

Vamos carregar alguns diálogos simples do conjunto de dados DialogSum Hugging Face. Esse conjunto de dados contém mais de 10.000 diálogos com os respectivos resumos e tópicos rotulados manualmente.

In [6]:
# huggingface_dataset_name = "knkarthick/dialogsum"
# dataset = load_dataset(huggingface_dataset_name)

In [ ]:
# print a couple of dialouges with theis baseline summaries
# Imprimir alguns diálogos com esses resumos de linha de base

examples_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(examples_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()


In [ ]:
#Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with .from_pretrained() method
#Carregue o modelo FLAN-T5, criando uma instância da classe AutoModelForSeq2SeqLM com o método .from_pretrained()

model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Para realizar a codificação e a decodificação, você precisa trabalhar com o texto em um formato tokenizado. A tokenização é o processo de dividir o texto em unidades menores que podem ser processadas pelos modelos LLM.


Faça o download do tokenizador para o modelo FLAN-T5 usando o método AutoTokenizer.from_pretrained(). O parâmetro use_fast ativa o tokenizador rápido. Nesta etapa, não há necessidade de entrar em detalhes, mas você pode encontrar os parâmetros do tokenizador na documentação.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

In [ ]:
#Test the tokenizer encoding and decoding a simple sentence:

sentence =  "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors = 'pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["inputs_ids"][0],
        skip_special_tokens = True
)

print('ENCODED SENTENCE:')
print(sentence_encoded["inputs_is"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

Agora é hora de explorar a capacidade do LLM base de resumir um diálogo sem nenhuma engenharia de prompt. A engenharia de promp é o ato de um ser humano alterar o prompt (entrada) para melhorar a resposta de uma determinada tarefa.

In [ ]:
for i, index in enumerate(examples_indices):
    dialogue = datasets['test'][index]['dialogue']
    summary = datasets['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors = 'pt')
    output = tokenizer.decode(
        model.generate(
            inputs["inputs_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens = True
    )

print(dash_line)
print('Example ', i + 1)
print(dash_line)
print('INPUT PROMPT:\n{dialogue}')
print(dash_line)
print('BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n`{output}\n')


Você pode ver que as suposições do modelo fazem algum sentido, mas ele não parece ter certeza da tarefa que deve realizar. Parece que ele apenas inventa a próxima frase do diálogo. A engenharia imediata pode ajudar aqui.

3 - Summarize Dialouge with as Instruction Prompt

3.1 - Zero Shot Inference with an Instruction Prompt

In [ ]:
for i, index in enumerate(examples_indices):
    dialogue = datasets['test'][index]['dialogue']
    summary = datasets['test'][index]['summary']

    prompt = f"""
    Summarize the follwuing conversation.

    {dialogue}

    Summary:
    """

    #Input constructed promp intead of dialogue
    inputs = tokenizer(prompt, return_tensors = 'pt')
    output = tokenizer.decode(
        model.generate(
            inputs["inputs_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens = True
    )

print(dash_line)
print('Example ', i + 1)
print(dash_line)
print('INPUT PROMPT:\n{dialogue}')
print(dash_line)
print('BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n`{output}\n')


3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5

In [ ]:
for i, index in enumerate(examples_indices):
    dialogue = datasets['test'][index]['dialogue']
    summary = datasets['test'][index]['summary']

    prompt = f"""
    Dialogue:

    {dialogue}

    What was going on?
    """

    #Input constructed promp intead of dialogue
    inputs = tokenizer(prompt, return_tensors = 'pt')
    output = tokenizer.decode(
        model.generate(
            inputs["inputs_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens = True
    )

print(dash_line)
print('Example ', i + 1)
print(dash_line)
print('INPUT PROMPT:\n{dialogue}')
print(dash_line)
print('BASELINE HUMAN SUMMARY:\n{summary}')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n`{output}\n')


4 - Summarize Dialouge with One Shot and Few Shot Inference
    - 4.1 - One Shot Inference
    - 4.2 Few Shot Inference

One shot e few shot inference é a prática de fornecer a um LLM um ou mais exemplos completos de pares de respostas a solicitações que correspondam à sua tarefa - antes da solicitação real que você deseja concluir. Isso é chamado de “in-context learning” e coloca seu modelo em um estado que compreende sua tarefa específica. Você pode ler mais sobre isso neste blog da HuggingFace.

In [ ]:
def make_prompt(example_indices_full, example_index_summarize):
    prompt

5 - Generative Configuration Parameters for Inference

Vamos criar uma função que receba uma lista de example_indices_full, gere um prompt com exemplos completos e, no final, anexe o prompt que você deseja que o modelo complete (example_index_to_summarize). Você usará o mesmo modelo de prompt do FLAN-T da seção 3.2.